In [2]:
import os
# os.getcwd()
os.chdir("C:\\Users\\DELL\\Desktop\\AnomalyDetectionChallenge")

In [3]:
import pandas as pd
from collections import Counter
import numpy as np
from statistics import mean, stdev
from math import sqrt

## if normal is stationary , anomaly is non-stationary, then anomaly! vice versia

In [5]:
data11 = pd.read_csv("hexacopter-hil-fifo-ls-01.kev.csv")
data12 = pd.read_csv("hexacopter-hil-fifo-ls-02.kev.csv")
data13 = pd.read_csv("hexacopter-hil-fifo-ls-sporadic.kev.csv")
data14 = pd.read_csv("hexacopter-hil-full-while.kev.csv")
data15 = pd.read_csv("hexacopter-hil-half-while.kev.csv")

In [40]:
# def:
def get_commonkey():
    
    setlist=[]  # take the common key accross all key levels(ONLY training)
    for i in range(1,11): 
        data=eval('data{}'.format(i))
        data["key"] = data["class"]+data["event"]
        setlist=setlist+[set(data["key"])]

    common_key=setlist[0]
    for i in range(1,len(setlist)):
        common_key=common_key&setlist[i]   # common_key get, length 83 key level across all 10 training files!
    return common_key

def gen_model_df(dataframe):
    newdata = []
    dataframe["key"] = dataframe["class"] + dataframe["event"]
    dataframe["numKey"] = dataframe["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
    dataframe = dataframe[dataframe.numKey != -1]
    dataframe["interval"] = dataframe["time"].diff()
    dataframe = dataframe[dataframe.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
    dataframe.index = pd.RangeIndex(len(dataframe.index))
    data_temp=dataframe[["key","numKey","interval"]]
    return data_temp


# def gen_concat_df(d1,d2):
#     df_0 = pd.concat([d1,d2])
#     return df_0


def interval_matrix(transitions,interval):  # return transition Matrix and the Matrix of events average time interval
    
    n = 1+ max(transitions)  # number of states
    M = [[0]*n for _ in range(n)] # transition matrix
    N = [[0]*n for _ in range(n)] # average time matrix
    X = [[0]*n for _ in range(n)] # standard deviation matrix
    u = 0
    for (i,j) in zip(transitions,transitions[1:]):
        u += 1
        M[i][j] += 1
        N[i][j] += interval[u]
        avg_local = N[i][j]/M[i][j]
        absdis = interval[u]-avg_local
        sdv_local = sqrt(absdis*absdis/(M[i][j]))
        X[i][j] = sdv_local
    
    a = np.array(N)
    b = np.array(M)
    N = np.divide(a, b, out = np.zeros_like(a), where = b!= 0) # take average time interval of each event sequence
        
    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    M = np.array(M)
    X = np.array(X)
    
    return M,N,X


def split_testing(data,N):  # split data onto many parts, return a list of data  
    datas=np.array_split(data, N)
    return datas


def run_model(model_input):
    transition_matrix, averagetime_matrix,std_matrix = interval_matrix(model_input["numKey"],model_input["interval"])
    return transition_matrix, averagetime_matrix,std_matrix



# training data:

In [4]:
# training: 
data1 = pd.read_csv("hexacopter-hil-clean-01.kev.csv")
data2 = pd.read_csv("hexacopter-hil-clean-02.kev.csv")
data3 = pd.read_csv("hexacopter-hil-clean-03.kev.csv")
data4 = pd.read_csv("hexacopter-hil-clean-04.kev.csv")
data5 = pd.read_csv("hexacopter-hil-clean-05.kev.csv")
data6 = pd.read_csv("hexacopter-hil-clean-06.kev.csv")
data7 = pd.read_csv("hexacopter-hil-clean-07.kev.csv")
data8 = pd.read_csv("hexacopter-hil-clean-08.kev.csv")
data9 = pd.read_csv("hexacopter-hil-clean-09.kev.csv")
data10 = pd.read_csv("hexacopter-hil-clean-10.kev.csv")



In [75]:
full_keyset = get_commonkey()
full_keyset = list(full_keyset)
length = len(full_keyset)


# def train_model():

result = []
avg_time = []
w_time = []

for i in range(1,11):  # dataset level training, 1-10 clean datasets
    
    df_newdata=eval('data{}'.format(i))
    
    split_df= split_testing(df_newdata,)
    
    for j in range(len(split_df)):
        
        input_df = split_df[j]
        df_model = gen_model_df(input_df)
        tm,am,sm = run_model(df_model)
        wti = tm*am
#         avg_time += [np.sum(am)/np.count_nonzero(am)]
        w_time += [np.sum(wti)/np.count_nonzero(wti)]
        

np.mean(w_time),np.std(w_time),np.max(w_time),np.min(w_time),np.ptp(w_time)


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [92]:
np.mean(w_time),np.std(w_time),np.max(w_time),np.min(w_time),np.ptp(w_time),len(w_time)

(71.06044297900065,
 5.422607572961724,
 84.70784362422121,
 53.46321571035906,
 31.244627913862153,
 500)

In [107]:
# def train model:

avg_time_t = []
w_time_t = []
anomaly_record = []
upper = np.mean(w_time)+3*np.std(w_time)
lower = np.mean(w_time)-3*np.std(w_time)

for i in range(11,16):  # dataset level training, 1-10 clean datasets
    
    df_newdata=eval('data{}'.format(i))
    
    split_df= split_testing(df_newdata,50)
    
    for j in range(len(split_df)):
        
        input_df = split_df[j]
        df_model = gen_model_df(input_df)
        tm,am,sm = run_model(df_model)
        wti_t = tm*am
        avg_wti_t = np.sum(wti)/np.count_nonzero(wti_t)
    
        w_time_t += [avg_wti_t]
        
        if avg_wti_t > upper or avg_wti_t < lower:
            anomaly_record += [(i,j)] 
        
        if len(anomaly_record)/len(w_time_t) > 0.05:
            print("anomaly detected! Location file:{0},fraction:{1}".format(i,j)) 
            break



C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


anomaly detected! Location file:11,fraction:0
anomaly detected! Location file:12,fraction:0
anomaly detected! Location file:13,fraction:0
anomaly detected! Location file:14,fraction:0
anomaly detected! Location file:15,fraction:0


In [89]:
np.mean(w_time_t),np.std(w_time_t),np.max(w_time_t),np.min(w_time_t),np.ptp(w_time_t),set(w_time_t)

(54.43794535374846,
 6.324280632734518,
 79.23131479430678,
 41.91478037109533,
 37.31653442321145,
 {41.91478037109533,
  43.0683981794741,
  43.2668700604855,
  43.46717964409886,
  43.56803156902717,
  43.66935257267607,
  43.77114593531634,
  43.976163012296745,
  44.497207597750496,
  44.60290167755513,
  44.709099062501686,
  44.81580335620694,
  44.923018196772034,
  45.03074725719595,
  45.24776290662822,
  45.35705701992925,
  45.46688040254409,
  45.577236908375504,
  45.91154426956164,
  46.024072564339974,
  46.36499162037212,
  46.47975645111562,
  46.595090834369,
  46.827485302370846,
  46.94455401562677,
  47.06220953947546,
  47.180456297112336,
  47.29929875629902,
  47.41874142992603,
  47.53878887658407,
  47.65944570114393,
  47.780716555345315,
  47.90260613839467,
  48.025119197572145,
  48.14826052884797,
  48.27203497750825,
  48.52150285852896,
  48.647206233810124,
  48.773562613638205,
  48.90057709961122,
  49.028254846607595,
  49.15660106348353,
  49.2856

In [104]:
[(2,3),(2)]

[(2, 3), 2]

In [7]:
## initial data processing, newdata contain 15 files, 10 training + 5 testing

setlist=[]  # take the common key accross all key levels(training+testing)
for i in range(1,16): 
    data=eval('data{}'.format(i))
    data["key"] = data["class"]+data["event"]
    setlist=setlist+[set(data["key"])]

common_key=setlist[0]
for i in range(1,11):
    common_key=common_key&setlist[i]   # common_key get, length 74 key level across all 15 files!
                                

newdata=[]
full_key_set = list(common_key) 
for i in range(1,16): 
    data=eval('data{}'.format(i))
    data["key"] = data["class"]+data["event"]
    data["numKey"] = data["key"].apply(lambda x: full_key_set.index(x) if x in full_key_set else -1)  # change string key to number key, unique to -1!
    data = data[data.numKey != -1]  
    data.sort_values("time", axis = 0, ascending = True, inplace = True)  # 0.05% data timestamp NOT correct!
    data["interval"] = data["time"].diff()
    data = data[data.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
    data.index = pd.RangeIndex(len(data.index))
    data_temp=data[["key","numKey","interval"]]
    newdata = newdata + [data_temp]

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
tm = []
atm = []
sdm = []

for i in range(len(newdata)):
    a,b,c = interval_matrix(newdata[i].numKey,newdata[i].interval)
    
    tm += [a]
    atm += [b]
    sdm += [c]

In [52]:
atm[1]

tmi= []


multi = []
sd = []
for i in range(len(atm)):
    itm = np.linalg.det(atm[i])
    multi += [itm]
    tmi += [sum(sum(atm[i]))/np.count_nonzero(atm[i])]
    sd += [sum(sum(sdm[i]))/(np.count_nonzero(sdm[i]))]
    

multi
mean(tmi)
# import matplotlib as plt

# # % matplotlin
# a= pd.DataFrame({"avg":tmi})
# a.plot()
# plt.show()

505.02140864567656

In [44]:
tst=data1[:50000]
set(tst.numKey)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 58,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81}

In [ ]:
def reward_matrix(transition): # reward matrix
    reward = np.array(transition)
    reward[reward != 0] = 1
    reward[reward == 0] = -1
    return reward



rwd_matrix()## 实时概率矩阵

def score(indicator, reward):   # return stability score
    indicator = np.array(indicator)
    reward = np.array(reward)
    score = sum(sum(indicator*reward))
    return score

In [6]:


# def generate_setlist(data):  # combine column "class" and "event"  into one column called "key"
    
#     setlist = []
#     data["key"] = data["class"]+data["event"]
#     setlist += list(set(data["key"]))
#     return setlist
    
# def find_commonkey(data1,data2):  # get common key of two datasets--Finally no common key in the processing and analysis , only use common keys in the training dataset!!
    
    
#     key1 = generate_setlist(data1)
#     key2 = generate_setlist(data2)
#     common_key = list(set(key1) & set(key2))
#     return common_key

def gen_concat_df(d1,d2):
    df_0 = pd.concat([d1,d2])
    return df_0


def gen_model_df(dataframe):
    newdata = []
    dataframe["key"] = dataframe["class"] + dataframe["event"]
    dataframe["numKey"] = dataframe["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
    dataframe = dataframe[dataframe.numKey != -1]
    dataframe["interval"] = dataframe["time"].diff()
    dataframe = dataframe[dataframe.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
    data_temp=dataframe[["key","numKey","interval"]]
    return data_temp




def concat_df(data1,data2):
    
    newdata=[]
    data0 = pd.concat([data1,data2])
    data0["key"] = data0["class"] + data0["event"]
    data0["numKey"] = data0["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
    data0 = data0[data0.numKey != -1]
    data0.sort_values("time", axis = 0, ascending = True, inplace = True)
    data0["interval"] = data0["time"].diff()
    data0 = data0[data0.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
#     data0.index = pd.RangeIndex(len(data0.index))
    data_temp=data0[["key","numKey","interval"]]
    return data_temp,data0




def interval_matrix(transitions,interval):  # return transition,avgtime,variance Matrix
    
    n = 1+ max(transitions)  # number of states
    M = [[0]*n for _ in range(n)] # transition matrix
    N = [[0]*n for _ in range(n)] # average time matrix
    X = [[0]*n for _ in range(n)] # standard deviation matrix
    u = 0

    for (i,j) in zip(transitions,transitions[1:]):
        u += 1
        M[i][j] += 1
        N[i][j] += interval[u]
        avg_local = N[i][j]/M[i][j]
        absdis = interval[u]-avg_local
        sdv_local = sqrt(absdis*absdis/(M[i][j]))
        X[i][j] = sdv_local
    
    a = np.array(N)
    b = np.array(M)
    N = np.divide(a, b, out = np.zeros_like(a), where = b!= 0) # take average time interval of each event sequence

    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
            
    M = np.array(M)
    X = np.array(X)
    return M,N,X





In [30]:
def gen_concat_df(d1,d2):
    df_0 = pd.concat([d1,d2])
    return df_0


def gen_model_df(dataframe):
    newdata = []
    dataframe["key"] = dataframe["class"] + dataframe["event"]
    dataframe["numKey"] = dataframe["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
    dataframe = dataframe[dataframe.numKey != -1]
    dataframe["interval"] = dataframe["time"].diff()
    dataframe = dataframe[dataframe.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
    data_temp=dataframe[["key","numKey","interval"]]
    return data_temp

In [32]:
df0 = gen_concat_df(data1,data2)
gen_model_df(df0)

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,key,numKey,interval
0,CONTROLBUFFER,25,NaN
2,PROCESSPROCCREATE_NAME,22,288.0
3,THREADTHCREATE,26,117.0
4,THREADTHREADY,60,30.0
5,THREADTHCREATE,26,53.0
...,...,...,...
685778,COMMSND_PULSE_EXE,78,29.0
685779,THREADTHREADY,60,27.0
685780,COMMSND_PULSE_EXE,78,32.0
685781,COMMREC_PULSE,66,150.0


In [13]:
def gen_vector(numKey):
    a = np.zeros(length)
    a[numKey] = 1
    return a

vector_dictionary = {i:gen_vector(i) for i in range(length)}
# k=vector_dictionary[1]
# sum(k == gen_vector(1))

def gen_vector_matrix(cc):   # create event matrix
    matrix = np.stack(map(lambda x: vector_dictionary[x], cc.numKey))
    matrix_T = matrix.T
    return matrix_T


In [8]:
# a=np.stack((vector_dictionary[0],vector_dictionary[1],vector_dictionary[2],vector_dictionary[3])).T
# tm2.dot(a)


cc0, datae = concat_df(data1, data2)

tm2, am2, sm2 = run_model(cc0)


cc1, datae = concat_df(datae, data3)

tm3, am3, sm3 = run_model(cc1)


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [9]:
cc15, dat15 = concat_df(datae, data15)
tm15, am15, sm15 = run_model(cc15)

cc14, dat14 = concat_df(datae, data14)
tm14, am14, sm14 = run_model(cc14)

cc13, dat13 = concat_df(datae, data13)
tm13, am13, sm13 = run_model(cc13)

cc12, dat12 = concat_df(datae, data12)
tm12, am12, sm12 = run_model(cc12)

cc11, dat11 = concat_df(datae, data11)
tm11, am11, sm11 = run_model(cc11)

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [29]:
np.linalg.det(am)

NameError: name 'am' is not defined

In [41]:
a=am2-am4
b=np.array([[1,0],[0,4]])-np.array([[0,0],[0,2]])
c=np.array([[1,0],[0,4]])
np.linalg.det(am4)

-1.0995617320288018e+76

In [16]:
kk1=np.stack(map(lambda x : vector_dictionary[x],cc0.numKey)).T

# kk2=np.stack(map(lambda x : vector_dictionary[x],cc1.numKey)).T

# kk3=np.stack(map(lambda x : vector_dictionary[x],cc2.numKey)).T

a1=np.linalg.matrix_power(tm2,10)
# a2=np.linalg.matrix_power(tm3,10)
# a3=np.linalg.matrix_power(tm4,10)


b1 = a1.dot(kk1)
# b2 = a2.dot(kk2)
# b3 = a3.dot(kk3)

r1 = np.argmax(b1,axis = 0)
# r2 = np.argmax(b2,axis = 0)
# r3 = np.argmax(b3,axis = 0)






# print(sum(r2[500000:600000]==cc1.numKey[500010:600010]))

# print(sum(r3[500000:600000]==cc2.numKey[500010:600010]))




C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


147
360
422


In [21]:
am4

array([[    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,   101.88235294],
       [   47.5       ,     0.        ,   365.75373134, ...,
          186.73484848,   362.65384615, 23577.6197411 ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,    83.43478261],
       ...,
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,   174.38655462],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,    83.81818182],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ]])

In [165]:
a1=np.linalg.matrix_power(tm2,200000)
a2=a1.dot(kk)
# np.amax(a,axis=0)[1:150]
xz=np.argmax(a2,axis=0)

b1=np.linalg.matrix_power(tm4,200000)
b2=b1.dot(kk)
# np.amax(a,axis=0)[1:150]
xz=np.argmax(a2,axis=0)

In [168]:
sum(xz[:100000]==cc0.numKey[100000:200000])

2719

In [153]:
cc0.numKey[10:210]

10     76
11     55
12     76
13     55
14     76
       ..
205     6
206    79
207    36
208    21
209    15
Name: numKey, Length: 200, dtype: int64

In [146]:
np.linalg.matrix_power(np.array([[3,1],[0,2]]),2)

array([[9, 5],
       [0, 4]])

In [33]:
# test=split_testing(data1,200)
# 
# cc0,data0 = concat_df(test[0],test[1])

# tm1,am1,sm1 = run_model(cc0)

# 用cc0 创造产生matrix的df， 用data0 来拼接下一个df
# cc0, data0 = concat_df(data0, test[2])

# cc0, data0 = concat_df(data1, data2)

# tm2, am2, sm2 = run_model(cc0)


cc1, data0 = concat_df(data0, data3)

tm3, am3, sm3 = run_model(cc1)


cc2, data0 = concat_df(data0, data15)


tm4, am4, sm4 = run_model(cc2)



C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [136]:
len(cc0)

1357157

In [139]:
k2=am4

In [141]:
k1[k1 == 0] = 0
k1[k1 != 0 ] = 1
k1.sum()

737.0

In [142]:
k2[k1 == 0] = 0
k2[k1 != 0 ] = 1
k2.sum()

737.0

In [ ]:
{1,2,3}+{2,4}

In [ ]:
# a=np.array(tm[1])*np.array(prob[1])
lst=[]
for i in range(len(newdata)):
    lst = lst+[sum(sum(tm[i]))]

import matplotlib.pyplot as plt

# plt.ylim(240000,350000) 
plt.xlim(0,17)

plt.plot(range(1,16),lst)

from statistics import mean, stdev

stdev(lst[:10]),mean(lst[:10])



In [ ]:
multipl=[]
for i in range(15):
    a=(lst[i]-mean(lst[:10]))/stdev(lst[:10])
    print(a)

In [ ]:
set([3])&set([2,3])

In [ ]:
common_key

In [ ]:
from math import sqrt
t=[2,1,0,2,1,2,0,2,2]
va=[None,100,0,2,1,2,0,2,100]

u = 0
v = 0

n = 1+ max(t)  # number of states
M = [[0]*n for _ in range(n)] # 
N = [[0]*n for _ in range(n)] #
X = [[0]*n for _ in range(n)]

for (i,j) in zip(t,t[1:]):
    u += 1
    M[i][j] += 1
    N[i][j] += va[u]
    avg_local = N[i][j]/M[i][j]
    sdv_local = sqrt((va[u]-avg_local)**2/(M[i][j]))
    X[i][j] = sdv_local

print(M)
print(N)   
print(X)

In [ ]:
import numpy as np

foo = np.array(X)
foo[foo != 0] = 1
foo[foo == 0] = -1
foo

In [ ]:
def split_testing(data,N):  # split data onto many parts, return a list of data
        
    datas=np.array_split(data, N)
    return datas

def generate_setlist(data):  # combine column "class" and "event"  into one column called "key"
    
    setlist = []
    data["key"] = data["class"]+data["event"]
    setlist += list(set(data["key"]))
    return setlist
    
def find_commonkey(data1,data2, testing = False):  # get common key of two datasets
    
    key1 = generate_setlist(data1)
    key2 = generate_setlist(data2)
    common_key = list(set(key1) | set(key2))
    return common_key

def concat_df(data1,data2,testing = False):
    
    newdata=[]
    full_keyset = find_commonkey(data1,data2)
    data0 = pd.concat([data1,data2])
    data0["key"] = data0["class"] + data0["event"]
    data0["numKey"] = data0["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
    data0 = data0[data0.numKey != -1]
    data0.sort_values("time", axis = 0, ascending = True, inplace = True)
    data0["interval"] = data0["time"].diff()
    data0 = data0[data0.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
    data0.index = pd.RangeIndex(len(data0.index))
    data_temp=data0[["key","numKey","interval"]]
    return data_temp,data0




def interval_matrix(transitions,interval):  # return transition,avgtime,variance Matrix
    
    n = 1+ max(transitions)  # number of states
    M = [[0]*n for _ in range(n)] # transition matrix
    N = [[0]*n for _ in range(n)] # average time matrix
    X = [[0]*n for _ in range(n)] # standard deviation matrix
    u = 0

    for (i,j) in zip(transitions,transitions[1:]):
        u += 1
        M[i][j] += 1
        N[i][j] += interval[u]
        avg_local = N[i][j]/M[i][j]
        absdis = interval[u]-avg_local
        sdv_local = sqrt(absdis*absdis/(M[i][j]))
        X[i][j] = sdv_local
    
    a = np.array(N)
    b = np.array(M)
    N = np.divide(a, b, out = np.zeros_like(a), where = b!= 0) # take average time interval of each event sequence

    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
            
    M = np.array(M)
    X = np.array(X)
    return M,N,X


def run_model(data):
    
    transition_matrix, averagetime_matrix,std_matrix = interval_matrix(data["numKey"],data["interval"])
    return transition_matrix, averagetime_matrix,std_matrix


In [81]:
def find_commonkey(data1,data2, testing = False):  # get common key of two datasets
    
    if testing == True:
        key = generate_setlist(data1)
        common_key = list(set(key))
        print("testing True")
    else:
        key1 = generate_setlist(data1)
        key2 = generate_setlist(data2)
        common_key = list(set(key1) | set(key2))
        print("000000")
        
    return common_key

def concat_df(data1,data2,testing = False):
    
#     newdata=[]
    if testing == True:
        full_keyset = find_commonkey(data1,data2,testing = True)
    else:
        full_keyset = find_commonkey(data1,data2)
    
    return full_keyset 

#     data0 = pd.concat([data1,data2])
#     data0["key"] = data0["class"] + data0["event"]
#     data0["numKey"] = data0["key"].apply(lambda x: full_keyset.index(x) if x in full_keyset else -1)  
#     data0 = data0[data0.numKey != -1]
#     data0.sort_values("time", axis = 0, ascending = True, inplace = True)
#     data0["interval"] = data0["time"].diff()
#     data0 = data0[data0.interval != 0]    # delete events that happen at same time(0.6% total records), current no better method!
#     data0.index = pd.RangeIndex(len(data0.index))
#     data_temp=data0[["key","numKey","interval"]]
#     return data_temp,data0



In [66]:
set(list(zip(cc0['numKey'], cc0['key'])))

{(0, 'PROCESSPROCTHREAD_NAME'),
 (1, 'INT_ENTR0x00000044'),
 (2, 'COMMREPLY_MESSAGE'),
 (3, 'CONTROLTIME'),
 (4, 'USREVENTEVENT-2'),
 (5, 'KER_CALLMSG_DELIVER_EVENT/21'),
 (6, 'COMMREC_PULSE'),
 (7, 'INT_HANDLER_EXIT0x00000049'),
 (8, 'INT_HANDLER_ENTR0x0000002e'),
 (9, 'INT_EXIT0x00000049'),
 (10, 'THREADTHCONDVAR'),
 (11, 'THREADTHREADY'),
 (12, 'THREADTHSIGWAITINFO'),
 (13, 'KER_CALLTIMER_TIMEOUT/75'),
 (14, 'KER_EXITMSG_CURRENT/10'),
 (15, 'COMMSND_PULSE_EXE'),
 (16, 'KER_EXITMSG_REPLYV/15'),
 (17, 'KER_CALLCONNECT_CLIENT_INFO/42'),
 (18, 'KER_CALLSIGNAL_WAITINFO/32'),
 (19, 'KER_EXITMSG_INFO/19'),
 (20, 'USREVENTEVENT-0'),
 (21, 'INT_HANDLER_ENTR0x00000029'),
 (22, 'KER_CALLMSG_ERROR/13'),
 (23, 'KER_EXITMSG_SENDV/11'),
 (24, 'KER_CALLMSG_SENDV/11'),
 (25, 'KER_CALLMSG_CURRENT/10'),
 (26, 'KER_CALLCONNECT_DETACH/40'),
 (27, 'INT_EXIT0x00000044'),
 (28, 'USREVENTEVENT-3'),
 (29, 'KER_CALLMSG_SENDVNC/12'),
 (30, 'INT_HANDLER_ENTR0x00000044'),
 (31, 'KER_CALLSYNC_CONDVAR_SIGNAL/83'),